# DomainCAT: Domain Connectivity Analysis Tool

### Analyzing the domain to domain connectivity of an Iris API Search

In [ ]:
# import all the helper functions and stuff
%run domain_cat_module.py

print("Notebook is ready to go")

## Load Domain Data From Iris Investigate API or a json File

In [ ]:
# Data Loading Config
api_username=""
api_key=""

query_api = False
save_search_to_disk = False
json_file_path = "data/dash_gov_dot_us.json"

search_hash = ""

if query_api:
    iris_results = query_domain_data(api_username, api_key, search_hash)
    iris_results = iris_results["response"]["results"]
    print(f'Iris API returned {len(iris_results)} domains')

    # save search results to disk to be used later
    if save_search_to_disk:
        with open(json_file_path, 'w') as f:
            json.dump(iris_results, f)
else:
    with open(json_file_path) as json_data:
        iris_results = json.loads(json_data.read())

    print(f'Loaded {len(iris_results)} domains from {json_file_path}')

## DomainCAT Configuration

Please refer to the DomainCAT documentation for details about these configuration options

In [ ]:
config = Config()

# config for pivoting on matching substrings. Only matching substrings this long or longer will be used to create a pivot
config.longest_common_substring = 6

# List of substrings to ignore when creating pivots by matching substrings
config.ignore_substrings = []

# use the pivot count to scale how important the pivot is during graph layout. Smaller pivot counts is more influence, and vice versa
config.scale_edge_strength_by_pivot_count = True

# Pivot count threshold. Any pivot with more than this value is discarded. sys.maxsize effectivly keeps all pivots
config.count_threshold = sys.maxsize

# The smallest pivot count size to use. Default of 2 means no pivots are filtered out because it's count is too low
config.min_pivot_size = 2

# theoretical max pivot size for calculating edge strengths
config.max_domains = 100000000

# If True DomainCAT will print out some debug info while building the connected graph of domains
config.print_debug_output = False

## Choose Which Pivots To Use & Build Domain Graph


In [ ]:
pivot_category_config = {
    "adsense",
    "google_analytics",
    "create_date",
    "redirect_domain",
    "registrar",  
    "ip_address",
    "ip_country_code",   
    "ip_isp",
    "ip_asn",
    "ssl_hash",
    "ssl_subject",
    "ssl_org",
    "ssl_email",
    
#     # Note: commented out ns_host and ns_ip because they double count ns connectedness when used with ns_domain. 
    "ns_domain",
#     "ns_host",  
#     "ns_ip",  
    
#     # Note: commented out mx_host and mx_ip because they double counts mx connectedness when used with mx_domain    
    "mx_domain",
#     "mx_host",
#     "mx_ip", 
    
    "tld",
    "longest_common_substring",
}

# Build the domain pivot graph structure
config.pivot_category_config = pivot_category_config
pivot_categories, domain_list, graph, trimmed_domains = build_domain_pivot_graph(iris_results, config)

## Trimmed Domains

In [ ]:
print_trimmed_domains = True
if print_trimmed_domains:
    if len(trimmed_domains["unconnected"]) > 0:
        print("trimmed unconnected domains:")
        for domain in trimmed_domains["unconnected"]: print(f"{domain.name}, risk {domain.risk_score}")
    if len(trimmed_domains["create_date"]) > 0:
        print("\ntrimmed domains with only create date pivot:")
        for domain in trimmed_domains["create_date"]: print(f"{domain.name}, risk {domain.risk_score}")

## Draw the Domain Graph in an Interactive 3D Layout

In [ ]:
build_3d_graph_layout(graph, domain_list)

In [ ]:
build_3d_graph_layout(graph, domain_list)

In [ ]:
build_3d_graph_layout(graph, domain_list)

## Calculate & Show Pivot Statistics

In [ ]:
# Calculate a bunch of pivot statistics to see how well connected all the domains in the search result are
calc_pivot_stats(pivot_categories, domain_list)

## Draw the Domain Graph in an Interactive 2D Layout

In [ ]:
# calculate the pivots shared in commmon across all selected domains
shared_pivots = {}
def get_2d_shared_pivots(domain_list, selected_domains):
    global shared_pivots
    shared_pivots = get_shared_pivots(domain_list, selected_domains)
    
build_2d_graph_layout(graph, domain_list, get_2d_shared_pivots)

## Heatmap of which pivots connect the most domains together: by pivot category

In [ ]:
if len(shared_pivots) == 0:
    print("Select a set of domains in the 2D graph")
else:
    create_pivot_heatmaps(shared_pivots)